In [1]:
import pandas as pd
import numpy as np

In [28]:
def fpre_process(fname):
    # Hot encoding for all categorical columns
    # Impute values for nulls in categorical and numerical columns

    # 1. Get the selected labels and features - Done
    # 2. Determine the cat / numerical in features - Done
    # 3. Determine columns with Nulls - Done
    # 4. Determine most frequent in Cat - Done
    # 5. Impute cat column values with most frequent - Done
    # 6. Detemine mean / average for Num - Done 
    # 7. Impute num column values with mean - Done
    # 8. Hot Encode - Done
    # 9. Standardize 
    # 10. Return X and y

    # for testing - Replace this code with extracting from label_features.py
    label = ['Survived']
    features = ['Pclass', 'Sex', 'Age', 'Embarked', 'Cabin']
    
    from base_script_eda import fload_csv
    
    df, size = fload_csv(fname)

    # calculate the numerical and cat columns

    df_bkp = df[features]
    cols = list(df[features].columns)
    num_cols = list(df[features]._get_numeric_data().columns)
    cat_cols = list(set(cols) - set(num_cols))
    
    
    # Determine columns with Null values and Replace NAs
    
    null_num_cols = df[num_cols].columns[df[num_cols].isnull().any()].tolist()
    null_cat_cols = df[cat_cols].columns[df[cat_cols].isnull().any()].tolist()
    #print null_num_cols, null_cat_cols
    
    # For numerical columns
    
    for col in null_num_cols:
        #print "Before " + str(df[col].isnull().sum())
        #df[col].replace(np.NaN, df[col].mean(), inplace=True)
        df[col].fillna(df[col].mean(), inplace=True)
        #print "After " + str(df[col].isnull().sum())
    
    # For categorical columns impute and hot encode
    
    for col in null_cat_cols:
        
        # impute missing values
        
        #print "Before " + str(df[col].isnull().sum())
        #df[col].replace(np.NaN, df[col].value_counts().max(), inplace=True)
        df[col].fillna(df[col].value_counts().max(), inplace=True)
        #print "After " +  str(df[col].isnull().sum())
        #print " Successfully Completed Null Imputation: " + str(col)
    
    for col in null_cat_cols:
        # Hot Encode 
        
        #print "Before encode: " + str(df.shape)
        df = pd.concat([df, pd.get_dummies(df[col])], axis=1)
        #print " Successfully Completed Hot Encode: " + str(col)
        #print "After encode: " + str(df.shape)
         

In [29]:
fpre_process('train.csv')

train.csv successfully loaded


In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.shape

In [ ]:
df['IsAlone'] = 0

In [ ]:
df.shape

In [ ]:
pd.show_versions(as_json=False)